In [3]:
import matplotlib.pyplot as plt
import numpy as np

import FaceToolKit as ftk
import DetectionToolKit as dtk




image_size = 160
v = ftk.Verification()
# Pre-load model for Verification
v.load_model("./models/20180204-160909/")
v.initial_input_output_tensors()


d = dtk.Detection()

Model directory: ./models/20180204-160909/
Metagraph file: model-20180204-160909.meta
Checkpoint file: model-20180204-160909.ckpt-266000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from ./models/20180204-160909/model-20180204-160909.ckpt-266000
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
from sklearn.cluster import DBSCAN
from imutils import build_montages
import numpy as np
import argparse
import pickle
import cv2

In [5]:
def img_to_encoding(img):
    aligned = d.align(img, False)[0]
    return v.img_to_encoding(aligned, image_size)

In [7]:
import os
images = []
image_names = []

for img in os.listdir('./faces/clare'):
    images.append(plt.imread('./faces/clare/' + img))
    image_names.append(img)


In [8]:
encodings = [img_to_encoding(img) for img in images]

In [9]:
clt = DBSCAN(eps=0.8, metric="euclidean", n_jobs=3)
clt.fit(encodings)

DBSCAN(algorithm='auto', eps=0.8, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=5, n_jobs=3, p=None)

In [10]:
clt

DBSCAN(algorithm='auto', eps=0.8, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=5, n_jobs=3, p=None)

In [11]:
clt.labels_

array([ 0,  0, -1, -1, -1, -1, -1, -1, -1,  0,  0,  0, -1, -1, -1, -1, -1,
        0,  0, -1,  0, -1, -1,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,
       -1,  0, -1,  0, -1, -1, -1, -1,  0, -1,  0,  0, -1, -1,  0,  0,  0,
        0, -1,  0,  0,  0, -1, -1,  0, -1, -1, -1,  0, -1,  0, -1, -1, -1,
       -1,  0, -1,  0,  0, -1, -1, -1,  0, -1,  0,  0])

In [12]:
len(images)

80

In [13]:
len(clt.labels_[clt.labels_ == 0])

39

In [14]:
# for i in range(len(clt.labels_)):
#     if clt.labels_[i] == 0:
#         plt.imshow(images[i])
#         plt.show()

In [17]:
import os
import logging
import matplotlib.pyplot as plt

logging.basicConfig(filename='pruning.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')


def prune_faces_from_dir(directory):
        
    if not os.path.exists('./cleaned'):
        os.makedirs('./cleaned')
        
    # dangerous!
    extracted_dir = os.path.join('./cleaned', directory.split('/')[-1])

    if not os.path.exists(extracted_dir):
        os.makedirs(extracted_dir)
    for parent ,__, files in os.walk(directory):
        num_faces = 0
        images = []
        image_names = []
        for file in files:
            images.append(plt.imread(parent + '/' + file))
            image_names.append(file)
        encodings = [img_to_encoding(img) for img in images]
  
        
        clt = DBSCAN(eps=0.8, metric="euclidean", n_jobs=3)
        clt.fit(encodings)
        relevant_images = [images[i] for i in range(len(images)) if clt.labels_[i] == 0]
        for i, face in enumerate(relevant_images):

            plt.imsave(os.path.join(extracted_dir, str(i) + '.jpg'), face)
#             except Exception as e:
#                 logging.error("Failed extraction for file: {}, {}".format(file, e))
#     logging.info("Extracted {} faces from {} directory".format(num_faces, parent))

In [18]:
prune_faces_from_dir('./faces/clare')